# Task
We want to achieve the following workflow:

Step 1. 
1. Write "Hello" in a file 

Step 2:
1. Check filename

**However, this is too simple for dflow. We want to write "Hello" in 10 different files.**

In [25]:
from typing import List

from dflow import Step, Workflow, argo_range
from dflow.python import OP, OPIO, Artifact, OPIOSign, PythonOPTemplate, Slices

In [11]:
class Hello(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'filename': str
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            'out_art': Artifact(str)
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        file_num = int(op_in["filename"].split('.')[0][1:])
        open(op_in["filename"], "w").write("Hello" * file_num)
        op_out = OPIO({
            'out_art': op_in["filename"]
        })
        return op_out

In [12]:
class Check(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'filename': Artifact(List[str])
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign()

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        print(op_in["filename"])
        return OPIO()

In [19]:
wf = Workflow("slices")
hello = Step("hello",
            PythonOPTemplate(Hello, image="python:3.8",
                            slices=Slices("{{item}}",
                                        input_parameter=["filename"],
                                        output_artifact=["out_art"]
                                        )
                            ),
            parameters={"filename": [f"f{x}.txt" for x in range(10)]},
            key="no1",
            with_param=argo_range(10))
check = Step("check",
            PythonOPTemplate(Check, image="python:3.8"),
            artifacts={"filename": hello.outputs.artifacts["out_art"]},
            )
wf.add(hello)
wf.add(check)
wf.submit();

Workflow has been submitted (ID: slices-h5bqp)
